In [3]:
import sys
sys.path.append('../../../build')
import numpy as np
import libry as ry

In [4]:
C = ry.Config()

C.addFile("../KOMO/switches/model2.g")
C.view()

In [5]:
komo = ry.KOMO()

komo.setModel(C, False)
komo.setTiming(2.5, 30, 5., 2)

In [6]:
komo.add_qControlObjective([], 2)

In [67]:
komo.addSquaredQuaternionNorms([], 3.)

In [68]:
#grasp
komo.addSwitch_stable(1., 2., "table", "gripper", "box")
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])#last is weight
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0.])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#slow - down - up
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([.9,1.1], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

#place
komo.addSwitch_stable(2., -1., "gripper", "table", "box")
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#slow - down - up
komo.addObjective([2.], ry.FS.qItself, [], ry.OT.eq, [], [], 1)
komo.addObjective([1.9,2.2], ry.FS.position, ["gripper"], ry.OT.eq, [], [0.,0.,.1], 2)

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Kin/switch.cpp:conv_times2tuples:38(-1) beyond the time!: endTime=2.2 phases=1



In [69]:
#komo.verbose = 4;
komo.optimize();
#/  komo.checkGradients();

komo.view(False, "result")

0

In [71]:
komo.view_play(False, .3)

0

## Same again, but only the keyframes:

In [10]:
komo=0

In [11]:
komo = ry.KOMO()
komo.setModel(C, False)
komo.setTiming(3., 1, 5., 1) #DIFFERENT

In [12]:
komo.add_qControlObjective([], 1, 1e-1) #DIFFERENT
komo.addSquaredQuaternionNorms([], 3.)

In [13]:
#grasp
komo.addSwitch_stable(1., 2., "table", "gripper", "box")
komo.addObjective([1.], ry.FS.positionDiff, ["gripper", "box"], ry.OT.eq, [1e2])
komo.addObjective([1.], ry.FS.scalarProductXX, ["gripper", "box"], ry.OT.eq, [1e2], [0.])
komo.addObjective([1.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

#DIFFERENT: UP-DOWN MISSING
    
#place
komo.addSwitch_stable(2., -1., "gripper", "table", "box")
komo.addObjective([2.], ry.FS.positionDiff, ["box", "table"], ry.OT.eq, [1e2], [0,0,.08])
komo.addObjective([2.], ry.FS.vectorZ, ["gripper"], ry.OT.eq, [1e2], [0., 0., 1.])

In [14]:
komo.optimize();
komo.view(False, "result")
print("constraints:", komo.getConstraintViolations())
print("costs:", komo.getCosts())
komo.getReport()

constraints: 8.627910530670535
costs: 0.021422584787208286


{'F_qItself/1-#16': {'order': 1.0, 'type': 'sos', 'sos': 0.021422584787208286},
 'F_qQuaternionNorms/0-#49': {'order': 0.0,
  'type': 'eq',
  'eq': 4.063217096117455e-09},
 'F_Pose/1-box': {'order': 1.0, 'type': 'eq', 'eq': 3.9216339534917233},
 'F_PoseRel/1-box-gripper': {'order': 1.0,
  'type': 'eq',
  'eq': 2.542656039997815e-06},
 'F_PositionDiff/0-gripper-box': {'order': 0.0,
  'type': 'eq',
  'eq': 4.5260441228212045e-07},
 'F_ScalarProduct/0-gripper-box': {'order': 0.0,
  'type': 'eq',
  'eq': 4.252642127333672e-08},
 'F_Vector/0-gripper': {'order': 0.0,
  'type': 'eq',
  'eq': 1.6101871175318143e-06},
 'F_qZeroVel/1-box': {'order': 1.0, 'type': 'eq', 'eq': 6.684237463327776e-07},
 'F_PositionDiff/0-box-table': {'order': 0.0,
  'type': 'eq',
  'eq': 0.7843136942190656},
 'sos': 0.021422584787208286,
 'ineq': 0.0,
 'eq': 8.627910530670535,
 'f': 0.0}

In [15]:
komo.view_play(False, .2)

0

In [16]:
C.view_close()
komo=0